## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In [2]:
#get the html data from the wiki link

data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [3]:
# Define the arrays for the colums and fill it with the data extracted from html
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')    
 
    if(len(cells) > 0):
        for cell in cells:
            postalCodeList.append(cell.find('b').text)
            if cell.find('i'):
                boroughList.append(cell.find('i').text)
                neighborhoodList.append('Not assigned')
            else:
                text_str = cell.find('span').text    
                if text_str.find('(') != -1:
                    separatorindex = text_str.index('(')
                    boroughList.append(cell.find('span').text[:separatorindex].replace('(','').replace(')',''))
                    neighborhoodList.append(cell.find('span').text[separatorindex:].replace('(','').replace(')',''))
                else:
                    boroughList.append(text_str)
                    neighborhoodList.append('Not assigned')
                
            
            
print(len(boroughList))
print(len(postalCodeList))
print(len(neighborhoodList))

180
180
180


In [4]:
# assign the array data to dictionary and then convert it to data frame

toronto_neighorhood = [('PostalCode', postalCodeList),
                      ('Borough', boroughList),
                      ('Neighborhood', neighborhoodList)]
toronto_df = pd.DataFrame.from_dict(dict(toronto_neighorhood))
toronto_df.head(15)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Queen's Park / Ontario Provincial Government,Not assigned
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [5]:
#remove the non assigned row

toronto_df = toronto_df[toronto_df.Borough != 'Not assigned']
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Queen's Park / Ontario Provincial Government,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don MillsNorth
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
#replace the / with , 
toronto_df = toronto_df.replace('/',',', regex=True)
toronto_df = toronto_df.reset_index(drop=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,"Queen's Park , Ontario Provincial Government",Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don MillsNorth
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
#shape of the dataframe
print(toronto_df.shape)

(103, 3)
